**Required**
* Connect to FHIR Synthetic Patient Data
* Connect to OHDSI Synthetic Patient Data
* Connect to HUSH+ Synthetic Patient Data

**Questions**
* What are potentially diagnostic cluster groups to inform drug selection?
* Are there differences in exposures among patients diagnosed with Asthma during childhood vs during adulthood?
* Are there differences in medications prescribed off-label by patients diagnosed with Asthma during childhood vs during adulthood?

In [18]:
## All the imports we need
from urllib2 import Request, urlopen, URLError
from urllib import quote,n quote_plus

import mysql.connector

import pprint, json, requests

from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from greentranslator.api import GreenTranslator

import dateutil

DISEASE2SYMPTOMS = [x.split("\t") for x in urlopen("https://www.nature.com/article-assets/npg/ncomms/2014/140626/ncomms5212/extref/ncomms5212-s4.txt").read().split("\n")]
DISEASE2SYMPTOMS = filter(lambda x: len(x) == 4, DISEASE2SYMPTOMS)
print DISEASE2SYMPTOMS[:5]

SyntaxError: invalid syntax (<ipython-input-18-9e491e56b827>, line 3)

In [2]:
#connect to UMLS
try:
    cnx = mysql.connector.connect(user='tadmin',
                                password='ncats_translator!',
                                database='umls',
                                host='translator.ceyknq0yekb3.us-east-1.rds.amazonaws.com')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    print
    #cnx.close()

In [3]:
#Define functions to use UMLS
def findICD10(name):
    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd10 = "Undef"
        for code in cursor:
            icd10 = code
        return (icd10[0])
    return ("Undef")

def findICD9(name):
    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD9CM' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd9 = "Undef"
        for code in cursor:
            icd9 = code
        return (icd9[0])
    return ("Undef")

**Question: What are potentially diagnostic cluster groups to inform drug selection?**
* Find patients diagnosed with Asthma
* Find symptoms for Asthma
* Find occurences of symptoms in Asthma patients
* Find symptom clusters among Asthma patients

In [4]:
## Given disease/condition term, get back ICD codes
def txt2icd(txt):
    url_con = "http://api.ohdsi.org/WebAPI/vocabulary/search"
    headers = {'content-type': 'application/json'}
    params = {"QUERY": "Asthma",
              "VOCABULARY_ID": ["ICD9CM"]
    }
    response = requests.post(url_con, data=json.dumps(params), headers=headers)
    data= json.loads(response.text.decode('utf-8'))
    icd9arr=[]
    for d in data:
        icd9arr.append(d["CONCEPT_CODE"])
    return icd9arr
print txt2icd('asthma')

[u'E945.7', u'493', u'493.9', u'493.90', u'493.92', u'493.91', u'493.2', u'493.20', u'493.22', u'493.21', u'493.82', u'493.0', u'493.00', u'493.02', u'493.01', u'V17.5', u'493.1', u'493.10', u'493.12', u'493.11', u'493.8', u'975.7']


In [5]:
#Find symptoms for Asthma
DISEASE2SYMPTOMS = [x.split("\t") for x in urlopen("https://www.nature.com/article-assets/npg/ncomms/2014/140626/ncomms5212/extref/ncomms5212-s4.txt").read().split("\n")]
DISEASE2SYMPTOMS = filter(lambda x: len(x) == 4, DISEASE2SYMPTOMS)
#print DISEASE2SYMPTOMS

In [6]:
# Find Asthma ICD10 codes
asthmaCodes = findICD10('Asthma')
#findICD10('Asthma')
findICD9('Fever')

u'780.60'

In [7]:
## Given disease name, get back symptoms (defined using MeSH terms) along with TFIDF scores
## Taken from https://www.nature.com/articles/ncomms5212
def disease2symptom(txt):
    s = filter(lambda x: txt.lower() in x[1].lower(), DISEASE2SYMPTOMS)
    return([(x[0], x[3]) for x in s])
disease = "Asthma"
symps = disease2symptom(disease)
print 'Found %s symptom MeSH terms for %s' % (len(symps), disease)

#for s in symps:
 #   findICD9(s)

Found 206 symptom MeSH terms for Asthma


In [8]:
## Pull JHU FHIR patients matching an ICD code
import requests
import urllib2
import json

#base_url = 'http://ictrweb.johnshopkins.edu/rest/synthetic'
#patients = base_url+"/Patient"+"?_count=50"

#req = requests.get(patients)

def findPatients(code):
    try:
        response = urllib2.urlopen("http://ictrweb.johnshopkins.edu/rest/synthetic/Condition?icd_10="+code+"")
    except:
        print "I can't find patients"
        exit()
    html = response.read()
    return (html)

In [9]:
## Pull HUSH+ patients matching a list of ICD codes
query = GreenTranslator ().get_query ()

HUSHpluspatients = query.clinical_get_patients (age='8', sex='male', 
                                        race='white', location='OUTPATIENT')
pprint.pprint (HUSHpluspatients[0])

Initializing thread provenance @thread: MainThread
{u'birth_date': u'2006-08-02 00:00:00',
 u'diag': {u'ICD10:B08.4': {u'2016-07-08 00:00:00': u'OUTPATIENT'},
           u'ICD10:E66.09': {u'2016-05-04 00:00:00': u'OUTPATIENT'},
           u'ICD10:F41.1': {u'2017-02-28 00:00:00': u'OUTPATIENT'},
           u'ICD10:F41.9': {u'2016-03-10 00:00:00': u'OUTPATIENT'},
           u'ICD10:F42.4': {u'2015-10-28 00:00:00': u'OUTPATIENT'},
           u'ICD10:F43.0': {u'2017-03-15 00:00:00': u'OUTPATIENT'},
           u'ICD10:F45.9': {u'2016-05-04 00:00:00': u'OUTPATIENT'},
           u'ICD10:F84.0': {u'2017-03-15 00:00:00': u'OUTPATIENT'},
           u'ICD10:F91.9': {u'2015-11-18 00:00:00': u'OUTPATIENT'},
           u'ICD10:H60.502': {u'2016-06-29 00:00:00': u'OUTPATIENT'},
           u'ICD10:H91.90': {u'2017-03-15 00:00:00': u'OUTPATIENT'},
           u'ICD10:H91.93': {u'2016-05-04 00:00:00': u'OUTPATIENT'},
           u'ICD10:J02.9': {u'2017-01-20 00:00:00': u'OUTPATIENT'},
           u'ICD10:J

**Question: Are there differences in exposures among patients diagnosed with Asthma during childhood vs during adulthood?**
* Find age of diagosis for Asthma patients
* Find zipcode for Asthma patients
* Find exposures by zipcode for Asthma patients
* Compare exposures of patients diagnosed with Asthma during childhood vs during adulthood

In [10]:
#Define functions needed to connect patients with exposures
def findAgeZipcode(stuff):
    res = []
    for p in stuff['entry']:
        d1 = p['assertedDate']
        id = p['subject']['reference'][8:]
        url = 'http://ictrweb.johnshopkins.edu/rest/synthetic/Patient/'+id
        txt = urlopen(url).read()
        st = json.loads(txt)
        #pprint.pprint(st)
        d2 = st['birthDate']
        zip = st['address'][0]['postalCode']
        d1 = dateutil.parser.parse(d1)
        d2 = dateutil.parser.parse(d2)
        #print d1
        #print d2
        diff = d1 - d2
        #print diff
        url = 'https://www.zipcodeapi.com/rest/lPMf5jmnBdclCZWkQwlFPJO6HkolG4N1TzgZSDnuRAPtzLOqi957STdzeBVVFIWz/multi-info.json/'+zip+'/degrees'
        txt = urlopen(url).read()
        st = json.loads(txt)
        #pprint.pprint(st)
        lat = st[zip]['lat']
        lng = st[zip]['lng']
        #print lat
        #print lng
        #print diff.days
        t = (id,d2,d1,diff.days,zip,lat,lng)
        res.append(t)
    return(res)

In [11]:
## Find age and zipcode for FHIR patients with asthma diagnoses
for c in asthmaCodes:
    
    ## Get asthma patients
    url = "http://ictrweb.johnshopkins.edu/rest/synthetic/Condition?icd_10="+c+"&_count=1000"
    txt = urlopen(url).read()
    stuff = json.loads(txt)
    pprint.pprint(findAgeZipcode(stuff)) 
    

HTTPError: HTTP Error 401: Unauthorized

**Question: Are there differences in medications prescribed off-label by patients diagnosed with Asthma during childhood vs during adulthood?**
* Find age of diagosis for Asthma patients
* Find medications taken by Asthma patients
* Find medication classes for medications taken by Asthma patients
* Compare medication classes taken by patients diagosed with Asthma during childhood vs during adulthood

In [12]:
## Pull meds for HUSH+ patients
meds = {}
for x in HUSHpluspatients:
    medList = x['medList']
    # Collect the unique meds
    for m in medList.keys():
        found = False
        try:
            meds[medList[m]] = meds[medList[m]]+1
        except KeyError:
            meds[medList[m]] = 1
pprint.pprint(meds)

{None: 4,
 u'24 HR Methylphenidate Hydrochloride 10 MG Extended Release Oral Capsule [Ritalin]': 1,
 u'ACYCLOVIR 5 % TOPICAL OINTMENT': 1,
 u'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION': 3,
 u'ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER': 7,
 u'AUGMENTIN ES-600 SUSPENSION': 3,
 u'AZITHROMYCIN 250 MG TABLET': 1,
 u'Acetaminophen 21.7 MG/ML / Hydrocodone Bitartrate 0.5 MG/ML Oral Solution': 1,
 u'Acetaminophen 32 MG/ML Oral Suspension': 1,
 u'Albuterol 0.83 MG/ML Inhalant Solution': 2,
 u'Amoxicillin 120 MG/ML / Clavulanate 8.58 MG/ML Oral Suspension': 2,
 u'Amoxicillin 80 MG/ML Oral Suspension': 8,
 u'Antipyrine 54 MG/ML / Benzocaine 14 MG/ML Otic Solution': 1,
 u'Azithromycin 40 MG/ML Oral Suspension': 10,
 u'BACTROBAN 2% OINTMENT': 1,
 u'BECLOMETHASONE DIPROPIONATE 40 MCG/ACTUATION AEROSOL INHALER': 3,
 u'CLARITHROMYCIN 250 MG TABLET': 1,
 u'CLEOCIN 75 MG/5 ML GRANULES': 1,
 u'CLONIDINE HCL ER 0.1 MG TABLET,EXTENDED RELEASE,12 HR': 4,
 u'CONCERTA ER 1

In [13]:
## For a given medication string get NCBO annotations
## We let NCBO match any ontology since just using RxNORM doesn't
## always give us just the drug name (e.g., "CLINDAMYCIN 15 MG/ML ORAL SOLUTION" is
## a valid RxNORM term)
def med2rxnorm(txt):
    url = 'http://data.bioontology.org/annotator?text=%s&apikey=b792dd1b-cdc2-4cc8-aaf2-4fa4fbf47e4e'
    try:
        txt = urlopen(url % quote_plus(txt)).read()
    except TypeError:
        print "Failed to open URL "+url
        txt = "[]"
    resp = json.loads(txt)
    if len(resp) == 0: return([])
    annos = []
    for aresp in resp:
        annos.extend([ x['text'] for x in aresp['annotations'] ])
    ##annos = filter(lambda x: not any(d in x for d in'0123456789'), annos)
    return(annos)
print med2rxnorm("CLINDAMYCIN 15 MG/ML ORAL SOLUTION")        


[u'CLINDAMYCIN 15 MG/ML ORAL SOLUTION', u'CLINDAMYCIN 15 MG/ML', u'CLINDAMYCIN', u'ORAL SOLUTION']


In [14]:
## Get RxNORM codes for medication strings
medrxnorm = {}
for med in meds.keys():
    if med is None: continue    
    annos = med2rxnorm(med)
    print 'Processing %s and found %d annotations' % (med, len(annos))
    medrxnorm[med] = {'count':meds[med], 'annos':annos}    

Processing TRIAMCINOLONE ACETONIDE 0.1 % TOPICAL OINTMENT and found 4 annotations
Processing Azithromycin 40 MG/ML Oral Suspension and found 4 annotations
Processing Trazodone Hydrochloride 50 MG Oral Tablet and found 5 annotations
Processing CONCERTA ER 27 MG TABLET and found 1 annotations
Processing NEOSTIGMINE METHYLSULFATE 0.5 MG/ML INJECTION SOLUTION and found 4 annotations
Processing CONCERTA ER 18 MG TABLET and found 1 annotations
Processing moxifloxacin 5 MG/ML Ophthalmic Solution and found 4 annotations
Processing BECLOMETHASONE DIPROPIONATE 40 MCG/ACTUATION AEROSOL INHALER and found 3 annotations
Processing Ofloxacin 3 MG/ML Otic Solution and found 4 annotations
Processing Diphenhydramine Hydrochloride 25 MG Oral Capsule and found 5 annotations
Processing Methylphenidate Hydrochloride 10 MG Oral Tablet and found 5 annotations
Processing ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER and found 3 annotations
Processing PROAIR HFA 90 MCG INHALER and found 1 annotations
P

In [21]:
## Given a drug (identified by RXNORM), get conditions that the drug has a 
## status (approved, phase 3) for, via NDK API
def drug2conditions(drug):
    import urllib
    url = "https://tripod.nih.gov/ndk/treatment/%s/conditions" % (urllib.quote(drug))
    page = urllib.urlopen(url).read().strip()
    if page == "":
        print "Failed to load "+url
        return []
    #print page
    try:
        resp = json.loads(page)
    except ValueError, HTTPError:
        print "Failed to parse result from "+url
        print page
        return []
    conds = []
    for aresp in resp:
        condname = aresp['name']
        if 'ICD10' in aresp.keys():
            condicd10 = aresp['ICD10']
        else: condicd10 = []
        conds.append( (aresp['status'], condname, condicd10) )
    return(conds)

In [22]:
keys = list(medrxnorm.keys())
n = 10
annos = medrxnorm[keys[n]]['annos']
conds = []
print annos
for anno in annos:
    conds.extend(drug2conditions(anno))
pprint.pprint(keys[n])
pprint.pprint(conds)
#"##", keys[n], "##", conds

[u'METHYLPHENIDATE HYDROCHLORIDE 10 MG ORAL TABLET', u'METHYLPHENIDATE HYDROCHLORIDE 10 MG', u'METHYLPHENIDATE HYDROCHLORIDE', u'METHYLPHENIDATE', u'ORAL TABLET']
Failed to load https://tripod.nih.gov/ndk/treatment/METHYLPHENIDATE%20HYDROCHLORIDE%2010%20MG%20ORAL%20TABLET/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/METHYLPHENIDATE%20HYDROCHLORIDE%2010%20MG/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ORAL%20TABLET/conditions
u'Methylphenidate Hydrochloride 10 MG Oral Tablet'
[(u'Approved', u'Attention-deficit hyperactivity disorders', [u'F90']),
 (u'Approved', u'Syndrome', []),
 (u'Approved', u'Narcolepsy', [u'G47.41']),
 (u'Approved', u'Emotional lability', [u'R45.86']),
 (u'Approved',
  u'Developmental disorder of scholastic skills, unspecified',
  [u'F81.9']),
 (u'Approved', u'CHILD syndrome', []),
 (u'Approved', u'Mental disorder, not otherwise specified', [u'F99']),
 (u'Approved', u'Child', []),
 (u'Approved', u'Brief reactive psychosis', []),


In [23]:
# Find off label prescriptions of medications
patientDrugInfo = {}
for p in HUSHpluspatients:
    print "Patient "+p['patient_id']
    diags = []
    #print "Diag codes"
    for d in p['diag']:
        if d[0:6]=="ICD10:":
            #print d
            diags.append(d[6:])
    med = []
    for m in p['medList'].keys():
        med.append(p['medList'][m])
    #print "Meds"
    #print med
    approved = {}
    phase4 = {}
    phase3 = {}
    phase2 = {}
    phase1 = {}
    unknownMeds = []
    for m in med:
        rxnorm = med2rxnorm(m)
        try:
            annos = medrxnorm[m]['annos']
        except KeyError:
            unknownMeds.append(m)
            annos = []
        #print "Approvals"
        #pprint.pprint(annos)
        for a in annos:
            #print a
            x = drug2conditions(a)
            #print "drug conditions"
            #print x
            for d in x:
                #print "Med info"
                #print d
                if d[0]=="Approved":
                    for c in d[2]:
                        try:
                            approved[m].append(c)
                        except KeyError:
                            approved[m] = [c]
                if d[0]=="Phase 4":
                    for c in d[2]:
                        try:
                            phase4[m].append(c)
                        except KeyError:
                            phase4[m] = [c]
                if d[0]=="Phase 3":
                    for c in d[2]:
                        try:
                            phase3[m].append(c)
                        except KeyError:
                            phase3[m] = [c]
                if d[0]=="Phase 2":
                    for c in d[2]:
                        try:
                            phase2[m].append(c)
                        except KeyError:
                            phase2[m] = [c]
                if d[0]=="Phase 1":
                    for c in d[2]:
                        try:
                            phase1[m].append(c)
                        except KeyError:
                            phase1[m] = [c]

        #conds.extend(x)
    #print "Results"
    #pprint.pprint(conds)
    #print "Diags"
    #pprint.pprint(diags)

    drugStatus = {}
    for m in p['medList'].keys():
        if m in unknownMeds:
            status = "Unknown"
        else:
            x = p['medList'][m]
            #print x
            try:
                a = approved[x]
            except KeyError:
                a = []
            try:
                p4 = phase4[x]
            except KeyError:
                p4 = []
            try:
                p3 = phase3[x]
            except KeyError:
                p3 = []
            try:
                p2 = phase2[x]
            except KeyError:
                p2 = []
            try:
                p1 = phase1[x]
            except KeyError:
                p1 = []

            if len(annos)==0:
                status = "Unknown"
            else:
                status = "offLabel"
            for d in diags:
                if d in p1:
                    status = "Phase1"
                if d in p2:
                    status = "Phase2"
                if d in p3:
                    status = "Phase3"
                if d in p4:
                    status = "Phase4"
                if d in a:
                    status = "Approved"
        drugStatus[x] = status
    patientDrugInfo[p['patient_id']] = drugStatus

print "Patient Drug info"
pprint.pprint(patientDrugInfo)
        

Patient 32227752
Failed to load https://tripod.nih.gov/ndk/treatment/OFLOXACIN%203%20MG/ML%20OTIC%20SOLUTION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/OFLOXACIN%203%20MG/ML/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/OTIC%20SOLUTION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/TRIAMCINOLONE%20ACETONIDE%201%20MG/ML%20TOPICAL%20CREAM/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/TRIAMCINOLONE%20ACETONIDE%201%20MG/ML/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/TOPICAL%20CREAM/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/CLINDAMYCIN%20150%20MG%20ORAL%20CAPSULE/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/CLINDAMYCIN%20150%20MG/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ORAL%20CAPSULE/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/TRIAMCINOLONE%20ACETONIDE%200.1%20%25%20TOPICAL%20OINTMENT/conditions
Failed to load https://tripo

Failed to load https://tripod.nih.gov/ndk/treatment/ALBUTEROL%200.83%20MG/ML%20INHALANT%20SOLUTION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ALBUTEROL%200.83%20MG/ML/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/INHALANT%20SOLUTION/conditions
Patient 37486050
Failed to load https://tripod.nih.gov/ndk/treatment/MOXIFLOXACIN%205%20MG/ML%20OPHTHALMIC%20SOLUTION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/MOXIFLOXACIN%205%20MG/ML/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/OPHTHALMIC%20SOLUTION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/SERTRALINE%2025%20MG/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/SERTRALINE%2025%20MG/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/MOXIFLOXACIN%205%20MG/ML%20OPHTHALMIC%20SOLUTION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/MOXIFLOXACIN%205%20MG/ML/conditions
Failed to load https://tripod.nih.gov/ndk/treatme

Failed to load https://tripod.nih.gov/ndk/treatment/AMOXICILLIN%2080%20MG/ML%20ORAL%20SUSPENSION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/AMOXICILLIN%2080%20MG/ML/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ORAL%20SUSPENSION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/AZITHROMYCIN%2040%20MG/ML%20ORAL%20SUSPENSION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/AZITHROMYCIN%2040%20MG/ML/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ORAL%20SUSPENSION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/AMOXICILLIN%20120%20MG/ML%20/%20CLAVULANATE%208.58%20MG/ML%20ORAL%20SUSPENSION/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/AMOXICILLIN%20120%20MG/ML/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/CLAVULANATE%208.58%20MG/ML/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ORAL%20SUSPENSION/conditions
Failed to load https://tripod.nih.gov/ndk/

Failed to load https://tripod.nih.gov/ndk/treatment/ATOMOXETINE%2010%20MG%20ORAL%20CAPSULE/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ATOMOXETINE%2010%20MG/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ORAL%20CAPSULE/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ATOMOXETINE%2010%20MG%20ORAL%20CAPSULE/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ATOMOXETINE%2010%20MG/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ORAL%20CAPSULE/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/LISDEXAMFETAMINE%20DIMESYLATE%2060%20MG%20ORAL%20CAPSULE/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/LISDEXAMFETAMINE%20DIMESYLATE%2060%20MG/conditions
Failed to load https://tripod.nih.gov/ndk/treatment/ORAL%20CAPSULE/conditions
Patient Drug info
{u'100900752': {u'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION': 'offLabel',
                u'ALBUTEROL SULFATE HFA 90 MCG/ACTUAT